In [1]:
import pandas as pd
import numpy as np 

In [16]:
data = pd.read_csv('./data/german_credit_data.csv').drop('Unnamed: 0', axis = 1)

In [17]:
data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


In [18]:
data_num = data.select_dtypes(include=np.number) #only numerical columns
data_cat = data.drop(data_num.columns, axis = 1) #only categorical columns

In [24]:
data.fillna('mean')

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,mean,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,mean,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,mean,1736,12,furniture/equipment
996,40,male,3,own,little,little,3857,30,car
997,38,male,2,own,little,mean,804,12,radio/TV
998,23,male,2,free,little,little,1845,45,radio/TV


In [32]:
for c in data_cat.columns:
    data_cat[c] = data_cat[c].fillna(data_cat[c].mode()[0])

In [33]:
data_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Sex               1000 non-null   object
 1   Housing           1000 non-null   object
 2   Saving accounts   1000 non-null   object
 3   Checking account  1000 non-null   object
 4   Purpose           1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [31]:
data_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Sex               1000 non-null   object
 1   Housing           1000 non-null   object
 2   Saving accounts   817 non-null    object
 3   Checking account  606 non-null    object
 4   Purpose           1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [28]:
data_cat.fillna(data_cat.mode())

,Sex,Housing,Saving accounts,Checking account,Purpose
0,male,own,little,little,radio/TV
1,female,own,little,moderate,radio/TV
2,male,own,little,NaN,education
3,male,free,little,little,furniture/equipment
4,male,free,little,little,car
...,...,...,...,...,...
995,female,own,little,NaN,furniture/equipment
996,male,own,little,little,car
997,male,own,little,NaN,radio/TV
998,male,free,little,little,radio/TV


In [57]:
data['date'] = '2002/08/31'

In [59]:
data['date'] = pd.to_datetime(data['date'], format = '%Y/%m/%d')

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Age               1000 non-null   int64         
 1   Sex               1000 non-null   object        
 2   Job               1000 non-null   int64         
 3   Housing           1000 non-null   object        
 4   Saving accounts   817 non-null    object        
 5   Checking account  606 non-null    object        
 6   Credit amount     1000 non-null   int64         
 7   Duration          1000 non-null   int64         
 8   Purpose           1000 non-null   object        
 9   date              1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 78.2+ KB


In [63]:
data_num.mean()

Age                35.546
Job                 1.904
Credit amount    3271.258
Duration           20.903
dtype: float64

In [75]:

pd.concat([data_cat,pd.get_dummies(data['Sex'])], axis = 1)

,Sex,Housing,Saving accounts,Checking account,Purpose,female,male
0,male,own,little,little,radio/TV,False,True
1,female,own,little,moderate,radio/TV,True,False
2,male,own,little,little,education,False,True
3,male,free,little,little,furniture/equipment,False,True
4,male,free,little,little,car,False,True
...,...,...,...,...,...,...,...
995,female,own,little,little,furniture/equipment,True,False
996,male,own,little,little,car,False,True
997,male,own,little,little,radio/TV,False,True
998,male,free,little,little,radio/TV,False,True


In [95]:
from preprocessing import preprocessing

In [96]:
preprocessing(data)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,date,Age,Job,Credit amount,Duration,Sex,Housing,Saving accounts,Checking account,Purpose,date
1,22,female,2,own,little,moderate,5951,48,radio/TV,2002-08-31,22,2,5951,48,female,own,little,moderate,radio/TV,2002-08-31
3,45,male,2,free,little,little,7882,42,furniture/equipment,2002-08-31,45,2,7882,42,male,free,little,little,furniture/equipment,2002-08-31
4,53,male,2,free,little,little,4870,24,car,2002-08-31,53,2,4870,24,male,free,little,little,car,2002-08-31
7,35,male,3,rent,little,moderate,6948,36,car,2002-08-31,35,3,6948,36,male,rent,little,moderate,car,2002-08-31
9,28,male,3,own,little,moderate,5234,30,car,2002-08-31,28,3,5234,30,male,own,little,moderate,car,2002-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,48,male,1,own,little,moderate,1743,24,radio/TV,2002-08-31,48,1,1743,24,male,own,little,moderate,radio/TV,2002-08-31
993,30,male,3,own,little,little,3959,36,furniture/equipment,2002-08-31,30,3,3959,36,male,own,little,little,furniture/equipment,2002-08-31
996,40,male,3,own,little,little,3857,30,car,2002-08-31,40,3,3857,30,male,own,little,little,car,2002-08-31
998,23,male,2,free,little,little,1845,45,radio/TV,2002-08-31,23,2,1845,45,male,free,little,little,radio/TV,2002-08-31


In [92]:
data_num = data.select_dtypes(include=np.number) #only numerical columns
data_cat = data.drop(data_num.columns, axis = 1)

In [94]:
data_cat

,Sex,Housing,Saving accounts,Checking account,Purpose,date
0,male,own,NaN,little,radio/TV,2002-08-31
1,female,own,little,moderate,radio/TV,2002-08-31
2,male,own,little,NaN,education,2002-08-31
3,male,free,little,little,furniture/equipment,2002-08-31
4,male,free,little,little,car,2002-08-31
...,...,...,...,...,...,...
995,female,own,little,NaN,furniture/equipment,2002-08-31
996,male,own,little,little,car,2002-08-31
997,male,own,little,NaN,radio/TV,2002-08-31
998,male,free,little,little,radio/TV,2002-08-31
